In [10]:
import os
from dotenv import load_dotenv, find_dotenv
import boto3
import psycopg2
import pandas as pd

In [11]:
env_path = find_dotenv()
load_dotenv(dotenv_path=env_path, override=True)

# control
print(".env chargé depuis :", env_path)
print("USERNAME =", os.getenv("USERNAME"))


.env chargé depuis : c:\Users\Floriane\Desktop\projets_Jedha\bloc1\recap\project_Kayak\.env
USERNAME = flodussart


In [12]:
AWS_ACCESS_KEY_ID = os.getenv('AWS_KEY')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_KEY')

DB_USER = os.getenv('USERNAME')
DB_PASS = os.getenv('PASSWORD')
DB_HOST = os.getenv('HOSTNAME')
DB_PORT = 5432
DB_NAME = os.getenv('DBNAME')

S3_BUCKET = 'flodussartprojectkayak'
S3_FILE_KEY = 'projectKayack/src/final_ter.csv'
LOCAL_FILE = 'hotels_weather_final_ter.csv'


In [13]:
# download csv
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='eu-north-1'
)
s3.download_file(S3_BUCKET, S3_FILE_KEY, LOCAL_FILE)
print(f"Fichier téléchargé : {LOCAL_FILE}")

Fichier téléchargé : hotels_weather_final_ter.csv


In [14]:
df = pd.read_csv(LOCAL_FILE, encoding='utf-8-sig')
df.head()

hotel_id                                         hotel_name  \
0         1                                     Aigues Marines   
1         2  Appartements 3 étoiles terrasse ou patio intra...   
2         3         Artemia Aigues-Mortes - Hotel avec piscine   
3         4                               Au Cœur des Remparts   
4         5                           Chez Céline et Sébastien   

                                           hotel_url  hotel_rating  \
0  https://www.booking.com/hotel/fr/aigues-marine...           9.5   
1  https://www.booking.com/hotel/fr/appartement-3...           9.6   
2  https://www.booking.com/hotel/fr/le-royal-hote...           9.1   
3  https://www.booking.com/hotel/fr/au-coeur-des-...           9.9   
4  https://www.booking.com/hotel/fr/chez-celine-e...           9.4   

                                   hotel_description  hotel_latitude  \
0  L’établissement Aigues Marines vous accueille ...       43.559720   
1  L’établissement Appartements 3 étoiles terrass...       43.565358   
2  Featuring free WiFi and a seasonal outdoor swi...       43.576396   
3  L’hébergement Au Cœur des Remparts se trouve à...       43.565401   
4  L’hébergement Chez Céline et Sébastien se situ...       43.570192   

   hotel_longitude  city_id      city_name  city_latitude  city_longitude  \
0         4.218698        1  Aigues-Mortes      43.566152         4.19154   
1         4.192750        1  Aigues-Mortes      43.566152         4.19154   
2         4.197818        1  Aigues-Mortes      43.566152         4.19154   
3         4.192973        1  Aigues-Mortes      43.566152         4.19154   
4         4.195081        1  Aigues-Mortes      43.566152         4.19154   

    temp_max   humidity  wind_speed     clouds  pop  good_weather_score  
0  19.544286  53.714286    5.901429  29.571429  1.0            6.610714  
1  19.544286  53.714286    5.901429  29.571429  1.0            6.610714  
2  19.544286  53.714286    5.901429  29.571429  1.0            6.610714  
3  19.544286  53.714286    5.901429  29.571429  1.0            6.610714  
4  19.544286  53.714286    5.901429  29.571429  1.0            6.610714

In [15]:
# Postgres connection
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)
cur = conn.cursor()

In [16]:
#truncate table
cur.execute("TRUNCATE TABLE hotels_weather;")
conn.commit()

In [17]:
#import data
with open(LOCAL_FILE, 'r', encoding='utf-8-sig') as f:
    next(f)
    cur.copy_expert("""
        COPY hotels_weather (
            hotel_id, hotel_name, hotel_url, hotel_rating, hotel_description,
            hotel_latitude, hotel_longitude, city_id, city_name,
            city_latitude, city_longitude, temp_max, humidity,
            wind_speed, clouds, pop, good_weather_score
        )
        FROM STDIN WITH CSV
    """, f)

In [18]:
conn.commit()
cur.close()
conn.close()